# Openning Data

In [8]:
using SparseArrays
using JuMP
using Ipopt
using GLPK
using LinearAlgebra
using MAT

In [9]:
file = matopen("R5T1.mat")
varnames = names(file)

4-element Array{String,1}:
 "K"
 "L"
 "S"
 "U"

In [10]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U")
K = read(file, "K")

10

In [11]:
(m,n)=size(S)
(r,c) = size(L)

(13094, 50)

# constructing model

In [12]:
mzeros = zeros(m,1);

In [13]:
function opt_vector0(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector0 (generic function with 1 method)

In [42]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    # @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b .>= S*v)
    @constraint(model, b .>= -S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(sqrt(1+b[j]) for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [47]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    # @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b .>= S*v)
    @constraint(model, b .>= -S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(b[j] for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [48]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-10
            indexes[k]= i
            k+=1
        end
    end
    return length(indexes[1:k-1])
end

zero_rows (generic function with 1 method)

In [49]:
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        #squares[i]=sqrt(z)
        squares[i]=z
    end
    return squares
end

row_squares (generic function with 1 method)

In [66]:
lambda = 50

50

In [67]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector(S,l,u,z,lambda)
zero_rows(v)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56862
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22768
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1270
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

12979

In [68]:
V=v;

In [69]:
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V)
    v = opt_vector(S,l,u,z,lambda)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
    print("***",zero_rows((S*V)'),"\n") 
    
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58212
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22831
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1333
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

**i : 3
***12939
***3
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56244
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22711
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1213
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

**i : 5
***12895
***5
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57320
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22829
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1331
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

**i : 7
***12856
***7
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56920
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22740
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1242
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

**i : 9
***12831
***9
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56786
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22781
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1283
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

...

**i : 41
***12592
***41
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57036
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22777
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1279
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alph

**i : 43
***12579
***43
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56526
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22757
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1259
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alph

**i : 45
***12569
***45
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56382
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22711
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1213
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alph

**i : 47
***12557
***47
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    55500
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22688
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1190
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alph

**i : 49
***12548
***49
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56480
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    22770
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1272
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alph

In [41]:
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V)
    v = opt_vector(S,l,u,z,lambda)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    58212
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22831
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1333
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

**i : 2
***12955
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    55932
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22703
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1205
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du al

**i : 4
***12909
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56988
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22791
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1293
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du al

**i : 5
***12895
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57320
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22829
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1331
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du al

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56894
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22765
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1267
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

...

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56382
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22711
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1213
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56510
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22755
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1257
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    55500
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22688
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1190
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    57234
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22789
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1291
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

**i : 49
***12548
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    56480
Number of nonzeros in Lagrangian Hessian.............:    21498

Total number of variables............................:    22770
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1272
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    42996
        inequality constraints with only lower bounds:    42996
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du a

In [70]:
V

13094×50 Array{Float64,2}:
  0.0          0.0           0.0          …   3.99121e-13   0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0          …  -4.97162e-13   0.0
  1.85484e-12  0.0           0.0              0.0           0.0
 -3.02313e-13  0.0           0.0              0.0           0.0
 -6.96163e-17  3.22785e-19  -1.55141e-18      0.0          -4.36534e-15
  3.95008e-13  0.0           0.0              0.0           0.0
  0.0          0.0           0.0          …   0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  ⋮                                       ⋱                
  0.0          0.0           0.0              0.0           0.0
  0.0    

In [71]:
n1 = zero_rows(V)
n2 = zero_rows((S*V)')
n1 ,n2

(12536, 50)

# writing to output file

In [72]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end